# Predicción de calidad del café

## Requerimientos

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from openpyxl import load_workbook

print("¡Herramientas listas!")


¡Herramientas listas!


## Carga de datos

In [127]:
# 📂 Leer rutas desde el archivo de texto
with open("rutas_archivos.txt", "r", encoding="utf-8") as f:
    rutas = [line.strip() for line in f if line.strip()]

print(f"Se encontraron {len(rutas)} archivos:")
for ruta in rutas:
    print("•", ruta)


Se encontraron 3 archivos:
• C:\Users\JOSE DANIEL M\OneDrive\Documentos\Universidad\Aprendizaje Estadístico\Tarea 1 Aprendizaje Automatico\CC FT 17   Formato de Control de Calidad Café de Trillado.xlsx
• C:\Users\JOSE DANIEL M\OneDrive\Documentos\Universidad\Aprendizaje Estadístico\Tarea 1 Aprendizaje Automatico\CC FT 18  Formato de  Tostión.xlsx
• C:\Users\JOSE DANIEL M\OneDrive\Documentos\Universidad\Aprendizaje Estadístico\Tarea 1 Aprendizaje Automatico\CC FT 21   Formato de Control de Despachos.xlsx


In [128]:
ruta_trillado, ruta_tostion, ruta_despacho = rutas
# rutas asigndas por separado

Hojas del primer archivo:

In [19]:
pd.ExcelFile(ruta_trillado).sheet_names

['CONTROL CALIDAD CAFE TRILLADO J',
 'Sheet2',
 'LOTES',
 'ACCIONES CORRECTIVAS   ']

Hojas del segundo archivo:

In [20]:
pd.ExcelFile(ruta_tostion).sheet_names

['TOSTIÓN JERICÓ L', 'TOSTIÓN JERICÓ', 'ACCIONES CORRECTIVAS ']

Hojas del tercer archivo:

In [126]:
pd.ExcelFile(ruta_despacho).sheet_names

['TOSTIÓN MEDELLÍN', 'TOSTIÓN JERICÓ', 'ACCIONES CORRECTIVAS ']

In [151]:
# Cargar cada base en su propio DataFrame
df_trillado_1 = pd.read_excel(ruta_trillado, engine="openpyxl", skiprows=8, header=None,sheet_name="CONTROL CALIDAD CAFE TRILLADO J" )
df_trillado_jerico = pd.read_excel(ruta_trillado, engine="openpyxl", skiprows=8, header=None, sheet_name="Sheet2")
df_tostion_1 = pd.read_excel(ruta_tostion, engine="openpyxl", skiprows=6, header=None, sheet_name= "TOSTIÓN JERICÓ L")
df_tostion_jerico= pd.read_excel(ruta_tostion, engine="openpyxl", skiprows=6, header=None, sheet_name= "TOSTIÓN JERICÓ")
df_despacho_medellin = pd.read_excel(ruta_despacho, engine="openpyxl", skiprows=6, header=0, sheet_name="TOSTIÓN MEDELLÍN", usecols=[0, 1, 2, 3, 4, 6, 9, 10, 11])
df_despacho_jerico = pd.read_excel(ruta_despacho, engine="openpyxl", skiprows=6, header=0, sheet_name="TOSTIÓN JERICÓ", usecols=[0, 1, 2, 3, 4, 6, 9, 10, 11])

#Etiquetado de columnas manual de Trillado
columnas_trillado = [
    "FECHA",
    "LOTE",
    "MARCA",
    "CANTIDAD",
    "HUMEDAD_%",
    "HUMEDAD_C/NC",
    "MALLAS_#",
    "MALLAS_C/NC",
    "VERIFICACION_FISICA",
    "NOTAS_CATACION",
    "PUNTAJE_#",
    "PUNTAJE_C/NC",
    "LIBERACION_LOTE",
    "RESPONSABLE"
]
columnas_tostion = [
    "FECHA",
    "LOTE",
    "ORIGEN",
    "VARIEDAD",
    "PROCESO",
    "BENEFICIO",
    "PESO_VERDE",
    "MERMA",
    "PESO_TOSTADO",
    "PERFIL",
    "TEMPER_INICIO-FINAL",
    "TIEMPO_TUESTE",
    "OBSERVACIONES",
    "RESPONSABLE"
]
columnas_despacho = [
    "PEDIDO",
    "FECHA_TUESTE",
    "FECHA_EMPAQUE",
    "MARCA",
    "PRESENTACION",
    "CANTIDAD",
    "CLIENTE",
    "RESPONSABLE",
    "VERIFICA"
]
df_trillado_1.columns = columnas_trillado
df_trillado_jerico.columns = columnas_trillado
df_tostion_1.columns = columnas_tostion
df_tostion_jerico.columns = columnas_tostion
df_despacho_medellin.columns = columnas_despacho
df_despacho_jerico.columns = columnas_despacho
print("Archivos cargados correctamente.")

Archivos cargados correctamente.


In [152]:
# Convertir las columnas de fecha a tipo datetime con formatos mixtos

# --- Trillado ---
df_trillado_1['FECHA'] = pd.to_datetime(
    df_trillado_1['FECHA'], 
    format='mixed', 
    dayfirst=True, 
    errors='coerce'
)

df_trillado_jerico['FECHA'] = pd.to_datetime(
    df_trillado_jerico['FECHA'], 
    format='mixed', 
    dayfirst=True, 
    errors='coerce'
)

# --- Tostión ---
df_tostion_1['FECHA'] = pd.to_datetime(
    df_tostion_1['FECHA'], 
    format='mixed', 
    dayfirst=True, 
    errors='coerce'
)

df_tostion_jerico['FECHA'] = pd.to_datetime(
    df_tostion_jerico['FECHA'], 
    format='mixed', 
    dayfirst=True, 
    errors='coerce'
)

# --- Despacho ---
df_despacho_medellin['FECHA_TUESTE'] = pd.to_datetime(
    df_despacho_medellin['FECHA_TUESTE'], 
    format='mixed', 
    dayfirst=True, 
    errors='coerce'
)

df_despacho_jerico['FECHA_TUESTE'] = pd.to_datetime(
    df_despacho_jerico['FECHA_TUESTE'], 
    format='mixed', 
    dayfirst=True, 
    errors='coerce'
)

df_despacho_medellin['FECHA_EMPAQUE'] = pd.to_datetime(
    df_despacho_medellin['FECHA_EMPAQUE'], 
    format='mixed', 
    dayfirst=True, 
    errors='coerce'
)

df_despacho_jerico['FECHA_EMPAQUE'] = pd.to_datetime(
    df_despacho_jerico['FECHA_EMPAQUE'], 
    format='mixed', 
    dayfirst=True, 
    errors='coerce'
)


In [153]:
#Separar la columna TEMPER_INICIO-FINAL en dos columnas TEM_INICIO y TEM_FINAL
#temp_series = df_tostion_1['TEMPER_INICIO-FINAL'].astype(str).str.replace('°', '', regex=False)
df_tostion_1[['TEM_INICIO', 'TEM_FINAL']] = df_tostion_1['TEMPER_INICIO-FINAL'].str.replace('°', '', regex=False).str.split('/', expand=True)
df_tostion_1.drop(columns=['TEMPER_INICIO-FINAL'], inplace=True)
df_tostion_jerico[['TEM_INICIO', 'TEM_FINAL']] = df_tostion_jerico['TEMPER_INICIO-FINAL'].str.replace('°', '').str.split('-', expand=True)
df_tostion_jerico.drop(columns=['TEMPER_INICIO-FINAL'], inplace=True)

Nos tenemos que aseguar que los dataframes estén recibiendo correctamente los datos de tipo numérico y no los estén pasando como un string

In [157]:
def clasificar_variables(df):
    resumen = {}

    for col in df.columns:
        tipo = df[col].dtype

        if pd.api.types.is_numeric_dtype(df[col]):
            resumen[col] = 'numérica'
        elif pd.api.types.is_bool_dtype(df[col]):
            resumen[col] = 'booleana'
        elif isinstance(tipo, pd.CategoricalDtype):
            resumen[col] = 'categórica'
        elif pd.api.types.is_string_dtype(df[col]):
            resumen[col] = 'string'
        elif pd.api.types.is_datetime64_any_dtype(df[col]):
            resumen[col] = 'fecha'
        else:
            resumen[col] = 'otros'

    return pd.Series(resumen, name='tipo_variable')


In [158]:
clasificar_variables(df_trillado_1)

FECHA                     fecha
LOTE                      otros
MARCA                     otros
CANTIDAD               numérica
HUMEDAD_%              numérica
HUMEDAD_C/NC              otros
MALLAS_#               numérica
MALLAS_C/NC               otros
VERIFICACION_FISICA       otros
NOTAS_CATACION            otros
PUNTAJE_#                 otros
PUNTAJE_C/NC              otros
LIBERACION_LOTE           otros
RESPONSABLE               otros
Name: tipo_variable, dtype: object

In [159]:
clasificar_variables(df_trillado_jerico)

FECHA                     fecha
LOTE                      otros
MARCA                     otros
CANTIDAD               numérica
HUMEDAD_%                 otros
HUMEDAD_C/NC              otros
MALLAS_#               numérica
MALLAS_C/NC               otros
VERIFICACION_FISICA       otros
NOTAS_CATACION            otros
PUNTAJE_#              numérica
PUNTAJE_C/NC              otros
LIBERACION_LOTE           otros
RESPONSABLE               otros
Name: tipo_variable, dtype: object

In [161]:
clasificar_variables(df_tostion_1)

FECHA               fecha
LOTE               string
ORIGEN             string
VARIEDAD           string
PROCESO            string
BENEFICIO          string
PESO_VERDE       numérica
MERMA            numérica
PESO_TOSTADO     numérica
PERFIL             string
TIEMPO_TUESTE       otros
OBSERVACIONES    numérica
RESPONSABLE        string
TEM_INICIO         string
TEM_FINAL           otros
Name: tipo_variable, dtype: object

In [162]:
clasificar_variables(df_despacho_jerico)

PEDIDO           numérica
FECHA_TUESTE        fecha
FECHA_EMPAQUE       fecha
MARCA              string
PRESENTACION       string
CANTIDAD         numérica
CLIENTE            string
RESPONSABLE        string
VERIFICA           string
Name: tipo_variable, dtype: object

In [163]:
def normalizar_presentacion_a_gramos(serie):
    """
    Convierte una serie de strings como '250 Gr', '0.5 KG', '20kg', etc.
    en su valor numérico en gramos como entero.
    """
    def convertir(valor):
        try:
            val = str(valor).strip().lower()
            if 'kg' in val:
                num = float(val.split()[0].replace('kg', '').strip())
                return int(num * 1000)
            elif 'gr' in val:
                num = float(val.split()[0].replace('gr', '').strip())
                return int(num)
            else:
                num = float(val.split()[0])
                return int(num)
        except:
            return None  

    return serie.apply(convertir)
# Aplicar la normalización a las columnas PRESENTACION
#Quitarle el "gr" a PRESENTACION
df_despacho_medellin['PRESENTACION'] = normalizar_presentacion_a_gramos(df_despacho_medellin['PRESENTACION'])

df_despacho_jerico['PRESENTACION'] = normalizar_presentacion_a_gramos(df_despacho_jerico['PRESENTACION'])

In [164]:
#Pasemos los datos a tipo numérico
df_tostion_1['TEM_INICIO'] = pd.to_numeric(df_tostion_1['TEM_INICIO'], errors='coerce')
df_tostion_1['TEM_FINAL'] = pd.to_numeric(df_tostion_1['TEM_FINAL'], errors='coerce')
df_tostion_jerico['TEM_INICIO'] = pd.to_numeric(df_tostion_jerico['TEM_INICIO'], errors='coerce')
df_tostion_jerico['TEM_FINAL'] = pd.to_numeric(df_tostion_jerico['TEM_FINAL'], errors='coerce')
df_despacho_medellin['PRESENTACION'] = pd.to_numeric(df_despacho_medellin['PRESENTACION'], errors='coerce')
df_despacho_jerico['PRESENTACION'] = pd.to_numeric(df_despacho_jerico['PRESENTACION'], errors='coerce')
df_despacho_medellin['PEDIDO'] = pd.to_numeric(df_despacho_medellin['PEDIDO'], errors='coerce')
df_despacho_jerico['PEDIDO'] = pd.to_numeric(df_despacho_jerico['PEDIDO'], errors='coerce')
df_trillado_1['MALLAS_#'] = pd.to_numeric(df_trillado_1['MALLAS_#'], errors='coerce')
df_trillado_jerico['MALLAS_#'] = pd.to_numeric(df_trillado_jerico['MALLAS_#'], errors='coerce')
df_trillado_1['PUNTAJE_#'] = pd.to_numeric(df_trillado_1['PUNTAJE_#'], errors='coerce')
df_trillado_jerico['PUNTAJE_#'] = pd.to_numeric(df_trillado_jerico['PUNTAJE_#'], errors='coerce')
df_trillado_1['HUMEDAD_%'] = pd.to_numeric(df_trillado_1['HUMEDAD_%'], errors='coerce')
df_trillado_jerico['HUMEDAD_%'] = pd.to_numeric(df_trillado_jerico['HUMEDAD_%'], errors='coerce')

Visualicemos las primeras filas de cada DataFrame para entender su estructura

In [165]:
# Función para inspección rápida de un DataFrame
def inspeccionar(nombre, df):
    print(f"\n📄 {nombre}: {df.shape[0]} filas, {df.shape[1]} columnas")
    display(df.head(5))
    print(df.columns.tolist())

# Inspección de cada uno
inspeccionar("Trillado Otros", df_trillado_1)
inspeccionar("Trillado Jericó", df_trillado_jerico)
inspeccionar("Tostión Otros", df_tostion_1)
inspeccionar("Tostión Jericó", df_tostion_jerico)
inspeccionar("Despacho Medellín", df_despacho_medellin)
inspeccionar("Despacho Jericó", df_despacho_jerico)


📄 Trillado Otros: 93 filas, 14 columnas


,FECHA,LOTE,MARCA,CANTIDAD,HUMEDAD_%,HUMEDAD_C/NC,MALLAS_#,MALLAS_C/NC,VERIFICACION_FISICA,NOTAS_CATACION,PUNTAJE_#,PUNTAJE_C/NC,LIBERACION_LOTE,RESPONSABLE
0,2022-07-19,01-190722,Madre Laura,765.00,10.9,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,C,Si,LFQ
1,2022-07-19,09-190722,Tabi Natural,204.00,10.2,C,14.0,C,C,"Frutas maduras, nibs de cacao, acidez brillant...",85.0,C,Si,LFQ
2,2022-07-19,10-190722,Don Mario,165.00,10.7,C,14.0,C,C,"Panela, durazno, miel, acidez brillante citric...",84.5,C,Si,LFQ
3,2022-07-27,07-19-07-22,Don Felix,0.45,10.5,C,14.0,C,C,"Moras maduras, chocolate negro, acidez media c...",84.5,C,Si,LFQ
4,2022-10-31,01-291022,Madre Laura,105.00,10.7,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,C,Si,LFQ


['FECHA', 'LOTE', 'MARCA', 'CANTIDAD', 'HUMEDAD_%', 'HUMEDAD_C/NC', 'MALLAS_#', 'MALLAS_C/NC', 'VERIFICACION_FISICA', 'NOTAS_CATACION', 'PUNTAJE_#', 'PUNTAJE_C/NC', 'LIBERACION_LOTE', 'RESPONSABLE']

📄 Trillado Jericó: 44 filas, 14 columnas


,FECHA,LOTE,MARCA,CANTIDAD,HUMEDAD_%,HUMEDAD_C/NC,MALLAS_#,MALLAS_C/NC,VERIFICACION_FISICA,NOTAS_CATACION,PUNTAJE_#,PUNTAJE_C/NC,LIBERACION_LOTE,RESPONSABLE
0,2022-08-30,01-300822,Madre Laura,32.0,NaN,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,C,Si,AC
1,2022-10-13,01-131022,Madre Laura,79.8,10.4,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,C,Si,AC
2,2022-10-18,01-181022,Madre Laura,38.0,10.5,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,C,SI,AC
3,2022-10-18,05-181022,Doña Dolly,43.0,10.0,C,14.0,C,C,"Chocolate dulce,fresas,miel,cuerpo cremoso,aci...",85.0,C,SI,AC
4,2022-10-29,01-291022,Madre Laura,272.0,10.5,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84.0,C,SI,AC


['FECHA', 'LOTE', 'MARCA', 'CANTIDAD', 'HUMEDAD_%', 'HUMEDAD_C/NC', 'MALLAS_#', 'MALLAS_C/NC', 'VERIFICACION_FISICA', 'NOTAS_CATACION', 'PUNTAJE_#', 'PUNTAJE_C/NC', 'LIBERACION_LOTE', 'RESPONSABLE']

📄 Tostión Otros: 499 filas, 15 columnas


...

['FECHA', 'LOTE', 'ORIGEN', 'VARIEDAD', 'PROCESO', 'BENEFICIO', 'PESO_VERDE', 'MERMA', 'PESO_TOSTADO', 'PERFIL', 'TIEMPO_TUESTE', 'OBSERVACIONES', 'RESPONSABLE', 'TEM_INICIO', 'TEM_FINAL']

📄 Despacho Medellín: 1583 filas, 9 columnas


,PEDIDO,FECHA_TUESTE,FECHA_EMPAQUE,MARCA,PRESENTACION,CANTIDAD,CLIENTE,RESPONSABLE,VERIFICA
0,564.0,2022-07-22,2022-07-25,Madre Laura,500,3,Natalia Londoño,LFQ,MS
1,564.0,2022-07-22,2022-07-25,Madre Laura,1000,1,Natalia Londoño,LFQ,MS
2,566.0,2022-07-22,2022-07-25,Madre Laura,2500,2,Mateo Duque,LFQ,MS
3,567.0,2022-07-22,2022-07-26,Madre Laura,2500,1,Akio,LFQ,MS
4,568.0,2022-07-25,2022-07-26,Madre Laura,2500,3,Juan Camilo Baez,LFQ,MS


['PEDIDO', 'FECHA_TUESTE', 'FECHA_EMPAQUE', 'MARCA', 'PRESENTACION', 'CANTIDAD', 'CLIENTE', 'RESPONSABLE', 'VERIFICA']

📄 Despacho Jericó: 27 filas, 9 columnas


,PEDIDO,FECHA_TUESTE,FECHA_EMPAQUE,MARCA,PRESENTACION,CANTIDAD,CLIENTE,RESPONSABLE,VERIFICA
0,618.0,2022-10-08,2022-10-08,Madre Laura,500,2,JAVIER VALENCIA,AC,DC
1,619.0,2022-10-08,2022-10-08,Madre Laura,500,1,BEATRIZ FRANCO,AC,DC
2,630.0,2022-10-08,2022-10-08,Madre Laura,500,1,M TERESA CADAVID,AC,DC
3,642.0,2022-10-08,2022-10-08,Madre Laura,500,2,M TERESA CADAVID,AC,DC
4,659.0,2022-10-08,2022-10-08,Madre Laura,500,1,BEATRIZ FRANCO,AC,DC


['PEDIDO', 'FECHA_TUESTE', 'FECHA_EMPAQUE', 'MARCA', 'PRESENTACION', 'CANTIDAD', 'CLIENTE', 'RESPONSABLE', 'VERIFICA']


## Exploración Inicial

Nuestra variable objetivo (Y) será PUNTAJE_#. Tras una primera inspección de las bases de datos, se identificaron variables que fueron descartadas del conjunto de entrada debido a que contienen información que se genera después del proceso de catación o que está directamente relacionada con la variable objetivo. Entre ellas se encuentran columnas como NOTAS_CATACION, PUNTAJE_C/NC, LIBERACION_LOTE u OBSERVACIONES, las cuales representan evaluaciones sensoriales o decisiones tomadas en función del resultado final. Hay otras variablles como PEDIDO Y LOTE que si bien se utilizan para conectar información entre tablas no deben hacer parte de las variables del modelo predictivo. Otras de las variables que no se utilizarán serán las booleanas C/NC que dependen de otra numérica, pues son redundantes y conocemos la regla de los parámetros, se encuntran en el pie de página de la tabla. Ellas son que el porcentaje de humedad esté por debajo de 11% y que el número de mallas esté por encima de 14*0.95.

Revisemos de las variables que no hemos descartado si sus varianzas no son nulas, ya que esto podría indicar que no aportan información relevante para el modelo. No revisamos las varianzas de los tiempos de las fechas porque este tipo de dato tampoco será un parámetro importante en el modelo, mas nos ayudará a ordenar el conjunto de entrenamiento y testeo.


In [166]:
columnas_trillado_num = [
    "CANTIDAD",
    "HUMEDAD_%",
    "MALLAS_#",
    "PUNTAJE_#"
]
columnas_tostion_num = [
    "PESO_VERDE",
    "MERMA",
    "PESO_TOSTADO",
    "TEM_INICIO",
    "TEM_FINAL"
    #"TIEMPO_TUESTE"
]
columnas_despacho_num = [
    "FECHA_TUESTE",
    "FECHA_EMPAQUE",
    "MARCA",
    "PRESENTACION",
    "CANTIDAD",
    "CLIENTE",
    "RESPONSABLE",
    "VERIFICA"
]
print("Varianzas Tostión")
#df_tostion_1[columnas_tostion_num].var().sort_values()
#df_tostion_jerico[columnas_tostion_num].var().sort_values()
print("\nVarianzas Trillado")
df_trillado_1[columnas_trillado_num].var().sort_values()
df_trillado_jerico[columnas_trillado_num].var().sort_values()
#print("\nVarianzas Despacho")
#df_despacho_medellin[columnas_despacho_num].var().sort_values()
#df_despacho_jerico[columnas_despacho_num].var().sort_values()

Varianzas Tostión

Varianzas Trillado


MALLAS_#        0.000000
HUMEDAD_%       0.062540
PUNTAJE_#       0.397849
CANTIDAD     5256.588909
dtype: float64

## Preprocesamiento

## Entrenamiento de Modelos

## Evaluación y Explicabilidad